# Cleaning of Scraped Data 

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
wdog_csv_df = pd.read_csv('wetdogfood.csv')
print(wdog_csv_df.shape)
wdog_csv_df.head()

(1290, 24)


,rating,list_price,weight,lifestage,protein,caloric,size,category,fibre,review_count,...,price,foodform,brand,fat,breedsize,specialdiet,benefits,product_list,maker,foodtexture
0,4.4,\n $23.88,NaN,"adult, all stages",9.5% min,"804 kcal/kg, 290 kcal/12.7 oz can",NaN,"Dog,Food,Wet Food",1.4% max,206.0,...,\n $21.24,wet food,NaN,3.5% min,"small breeds, medium breeds, large breeds, all...","grain-free, no corn no wheat no soy","<li>the best, healthy ingredients from the ear...",whole earth farms grain-free hearty beef stew ...,By Whole Earth Farms,stew
1,4.6,NaN,NaN,adult,2.5% min,330 kcal/12.5 oz can,"12.5-oz, case of 12","Dog,Food,Wet Food",2.0% max,175.0,...,\n $35.99,wet food,"hill's, science diet",3.5% min,"all breeds, small breeds, medium breeds, large...",prescription food,<li>protects vital kidney and heart function w...,hill's prescription diet k/d kidney care chick...,By Hill's Prescription Diet,stew
2,4.9,\n $22.99,NaN,adult,4.5% min,329 kcal/12.8 oz can,NaN,"Dog,Food,Wet Food",1.5% max,40.0,...,\n $18.85,wet food,hill's,2.8% min,"all breeds, small breeds, medium breeds, large...",NaN,"<li>this recipe is made with real meat, brown ...",hill's science diet adult savory stew with bee...,By Hill's Science Diet,stew
3,4.6,\n $27.99,NaN,adult,10.0% min,"1,425 kcal/kg, 506 kcal/can",NaN,"Dog,Food,Wet Food",1.5% max,97.0,...,\n $25.99,wet food,blue wilderness,9.0% min,"all breeds, small breeds, medium breeds, large...","grain-free, high-protein, no corn no wheat no soy","<li>made with the finest natural ingredients, ...",blue buffalo wilderness duck & chicken grill g...,By Blue Buffalo,pate
4,4.6,\n $19.08,NaN,adult,8.0% min,"954 kcal/kg, 338 kcal/can","12.5-oz, case of 12","Dog,Food,Wet Food",1.0% max,33.0,...,\n $18.22,wet food,nutro,5.5% min,"large breeds, giant breeds",NaN,<li>real beef and beef liver in a savory broth...,nutro max large breed adult beef & rice dinner...,By Nutro Max,chunks in gravy


In [3]:
wetdog_df=wdog_csv_df[['price','weight','size','product_list']]
wetdog_df.head()
#Check that all Prices include '$'
check_price=all(wetdog_df.price.map(lambda p:bool(re.search('$',str(p)))))
if(check_price):
    wetdog_df['price']=wetdog_df.price.str.replace('\n\s+\$','').str.strip(' ')
else:
    raise('Price Columns incomplete')
wetdog_df.head(12)


C:\Users\cmlim\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,price,weight,size,product_list
0,21.24,NaN,NaN,whole earth farms grain-free hearty beef stew ...
1,35.99,NaN,"12.5-oz, case of 12",hill's prescription diet k/d kidney care chick...
2,18.85,NaN,NaN,hill's science diet adult savory stew with bee...
3,25.99,NaN,NaN,blue buffalo wilderness duck & chicken grill g...
4,18.22,NaN,"12.5-oz, case of 12",nutro max large breed adult beef & rice dinner...
5,33.99,NaN,NaN,"wellness 95% beef grain-free canned dog food,1..."
6,30.99,NaN,NaN,purina pro plan veterinary diets nf kidney fun...
7,15.20,NaN,NaN,purina one smartblend tender cuts in gravy chi...
8,18.96,NaN,NaN,purina pro plan focus adult 7+ beef & rice ent...
9,24.54,NaN,"12.7-oz, case of 12",newman's own organics grain-free 95% chicken &...


## Collate Weight Column
Product weight locations lack in uniformity (data structure of website).
Only a handful products have the weight data correctly stored.  
- Most are located within the Product List columns
- A handful are within the Size Columns

In [4]:
wetdog_df2=wetdog_df.copy()
wetdog_df2=wetdog_df2.fillna(0) #Fill NAs with Zero
wetdog_df2.head()

,price,weight,size,product_list
0,21.24,0,0,whole earth farms grain-free hearty beef stew ...
1,35.99,0,"12.5-oz, case of 12",hill's prescription diet k/d kidney care chick...
2,18.85,0,0,hill's science diet adult savory stew with bee...
3,25.99,0,0,blue buffalo wilderness duck & chicken grill g...
4,18.22,0,"12.5-oz, case of 12",nutro max large breed adult beef & rice dinner...


### Using Regex Search & Match 
Detect and Include cases of different Keywords ('tub','tray','pounds','oz-case') 
### Calculate Total Weight (lb)
Weight(lb)= Oz-per-can * Case Size  

In [5]:
#### Extract Weight & Size ####
#Add Weight and ProductList

for index, row in wetdog_df2.iterrows():
    #print(row['product_list'])
    row['product_list']=re.sub('oz tub,','oz,',row['product_list'])
    row['product_list']=re.sub('oz tray,','oz,',row['product_list'])
    row['product_list']=re.sub('oz case','oz,case',row['product_list'])
    regex=',(\S+)[-| ]oz,case of (\d+)'
    
    #regex_tray=',(\S+)[-| ]oz tray,case of (\d+)'
    if (row['weight']!=0):
        mpd=re.search('(\S+).*pounds', row['weight'])
        moz=re.search('(\S+).*ounces', row['weight'])
        if(bool(mpd)):
            row['weight']=float(mpd.group(1))
        if(bool(moz)):
            row['weight']=float(moz.group(1))*0.0625
    elif((row['size'])!=0):
        regex1='(\S+)-oz, case of (\d+)'
        regex2='(\S+)-lb'
        m1=re.search(regex1,row['size'])
        m2=re.search(regex2,row['size'])
        if(bool(m1)):
            canwt=float(m1.group(1))
            cans=int(m1.group(2))
            row['weight']=canwt*cans*0.0625
        elif(bool(m2)):
            row['weight']=float(m2.group(1))
    else:
        #regex=',(\S+)[-| ]oz,case of (\d+)'
        m=re.search(regex,row['product_list'])
        if(bool(m)):
            canwt=float(m.group(1))
            cans=int(m.group(2))
            row['weight']=canwt*cans*0.0625
        
    regex2='canned dog food'
    row['product_list']=re.sub(regex,'',row['product_list'])
    row['product_list']=re.sub(regex2,'',row['product_list'])
    
wetdog_df2=wetdog_df2.drop('size',axis=1)
wetdog_df2.head()

,price,weight,product_list
0,21.24,9.525,whole earth farms grain-free hearty beef stew
1,35.99,9.375,hill's prescription diet k/d kidney care chick...
2,18.85,9.6,hill's science diet adult savory stew with bee...
3,25.99,9.375,blue buffalo wilderness duck & chicken grill g...
4,18.22,9.375,nutro max large breed adult beef & rice dinner...


### Clean Maker Column & Generate Brand Column
- Remove Prefix 'By ' in maker column
- Consolidate Makers with Multiple Sub-Range to a Single Brand

In [6]:

wetdog_df2['maker']=wdog_csv_df.maker.map(lambda m: re.sub('By ','',m))

def detect_brand(brand):
    group=['Hill\'s', 'Purina', 'Nutro','Royal Canin']
    brandout=brand
    for grp in group:
        m=re.search(grp,brand)
        if(bool(m)):
           brandout=grp
    return(brandout)
wetdog_df2['brand']=wetdog_df2.maker.map(detect_brand,m)
print(wetdog_df2.brand.unique())
wetdog_df2.head()


['Whole Earth Farms' "Hill's" 'Blue Buffalo' 'Nutro' 'Wellness' 'Purina'
 "Newman's Own Organics" 'Royal Canin' 'Holistic Select' 'Pedigree'
 "Evanger's" 'Dogswell' 'Cesar' "Nature's Logic" 'Nulo' 'Health Extension'
 'Weruva' 'Eukanuba' 'Natural Balance' 'Instinct' 'Castor & Pollux'
 'Eagle Pack' 'Party Animal' 'Merrick' 'ALPO' "Nature's Recipe" 'Iams'
 "Dave's Pet Food" 'CANIDAE' 'PetKind' 'Halo' 'EVO' 'Wild Calling'
 'Mighty Dog' 'Zignature' 'Chicken Soup' 'Solid Gold' 'Redbarn'
 'Earthborn Holistic' 'Caru' 'Rachael Ray Nutrish' 'Goody Box'
 'Taste of the Wild' "Kibbles 'n Bits" 'Precise Holistic Complete'
 'Tiki Dog' 'Tender & True' 'KASIKS' 'Variety Pet Foods' 'Triumph'
 'K9 Natural' 'Go!' 'AvoDerm' 'Bravo!' 'PetGuard' 'FirstMate'
 'Precise Naturals' 'Canine Caviar' 'Nutrisca' 'Under the Sun' 'Evolve'
 'Hound & Gatos' 'HI-TOR' 'Lotus' 'Wysong' 'Addiction' 'I and Love and You'
 'Ziwi' 'Gentle Giants' 'Victor' 'Three Dog Bakery' 'Against the Grain'
 'Tuscan Natural' 'Almo Nature']


,price,weight,product_list,maker,brand
0,21.24,9.525,whole earth farms grain-free hearty beef stew,Whole Earth Farms,Whole Earth Farms
1,35.99,9.375,hill's prescription diet k/d kidney care chick...,Hill's Prescription Diet,Hill's
2,18.85,9.6,hill's science diet adult savory stew with bee...,Hill's Science Diet,Hill's
3,25.99,9.375,blue buffalo wilderness duck & chicken grill g...,Blue Buffalo,Blue Buffalo
4,18.22,9.375,nutro max large breed adult beef & rice dinner...,Nutro Max,Nutro


## Copy over Columns : Rating, Review Count, Food Texture, ID 

In [7]:
wetdog_df2['rating']=wdog_csv_df['rating']
wetdog_df2['review_cnt']=wdog_csv_df['review_count']
wetdog_df2['foodtexture']=wdog_csv_df['foodtexture']
wetdog_df2['id']=wdog_csv_df['itemnumber']
print(wetdog_df2.foodtexture.unique())
wetdog_df2.head()

['stew' 'pate' 'chunks in gravy' 'minced' 'bits in broth'
 'pate, chunks in gravy' 'shredded' 'flaked' 'stew, chunks in gravy' nan
 'bits in broth, pate' 'sliced' 'aspic / gelee' 'ground' 'minced, shredded'
 'grilled']


,price,weight,product_list,maker,brand,rating,review_cnt,foodtexture,id
0,21.24,9.525,whole earth farms grain-free hearty beef stew,Whole Earth Farms,Whole Earth Farms,4.4,206.0,stew,73955
1,35.99,9.375,hill's prescription diet k/d kidney care chick...,Hill's Prescription Diet,Hill's,4.6,175.0,stew,82589
2,18.85,9.6,hill's science diet adult savory stew with bee...,Hill's Science Diet,Hill's,4.9,40.0,stew,52789
3,25.99,9.375,blue buffalo wilderness duck & chicken grill g...,Blue Buffalo,Blue Buffalo,4.6,97.0,pate,85742
4,18.22,9.375,nutro max large breed adult beef & rice dinner...,Nutro Max,Nutro,4.6,33.0,chunks in gravy,60626


## Save Cleaned Dataset for Data Analysis & Visualization

In [90]:
print(wetdog_df2.shape)
wetdog_df2 = wetdog_df2[wetdog_df2.weight != 0]
print(wetdog_df2.shape)
wetdog_df2.to_csv('wetdogfood_clean.csv')

(1262, 9)
(1262, 9)


# Prepping of Scraped Review Data

In [21]:
wetdogfood_review_df = pd.read_csv('review_wetdogfood.csv')#,encoding="ISO-8859-1",index_col=0)
wetdogfood_review_df.head()

,category,review_full_content_0,review_full_content_1,pos_review_title,pos_review_content,itemnumber,neg_review_content,maker,neg_review_title
0,"Dog,Food,Wet Food",It is not as stew-y as I expected based on oth...,I was surprised my two large dogs didn't like ...,My girls LOVE this food!!!!,It is not as stew-y as I expected based on oth...,73955,I was surprised my two large dogs didn't like ...,By Whole Earth Farms,Surprising mediocre response from my dogs
1,"Dog,Food,Wet Food","I have two small, picky-eaters and had good lu...",NaN,Excellent as a topper for dry kibble,"I have two small, picky-eaters and had good lu...",50640,Selected this food because of the nutritional ...,By Taste of the Wild,Didn't do the trick
2,"Dog,Food,Wet Food",NaN,NaN,My dogs ate this up!,"My dogs are really finicky, but they lapped th...",60626,This Was supposed to have Rice with Beef Chunk...,By Nutro Max,Nutro Max Large Breed Adult Beef & Rice Dinner...
3,"Dog,Food,Wet Food",I had tried another type and brand of this and...,I used to buy one of these rolls every few mon...,What a terrific product,I had tried another type and brand of this and...,76564,I used to buy one of these rolls every few mon...,By Natural Balance,Not the Natural Balance roll of a few years ago
4,"Dog,Food,Wet Food",NaN,NaN,My dog loves this,I have not tasted it myself but my Mastiff rea...,84997,Except that this particular variety gave her d...,By Blue Buffalo,"Dog Liked It, But"


In [25]:
pos_review_df=wetdogfood_review_df[['pos_review_content']]
pos_review_df.to_csv('pos_review.csv')
pos_review_df=wetdogfood_review_df[['neg_review_content']]
pos_review_df.to_csv('neg_review.csv')

In [26]:
pos_title_df=wetdogfood_review_df[['pos_review_title']]
pos_title_df.to_csv('pos_title.csv')
pos_title_df=wetdogfood_review_df[['neg_review_title']]
pos_title_df.to_csv('neg_title.csv')

## Clean Key Benefits column

In [24]:
wdog_csv_df = pd.read_csv('wetdogfood.csv')
wdog_csv_df.head()
wetdog_benefits_df=wdog_csv_df[['benefits']]
wetdog_benefits_df.head()
wetdog_benefits_df['benefits']=wetdog_benefits_df.benefits.map(lambda lines:re.sub('<\S{1,3}>',' ',lines))
wetdog_benefits_df.head()
wetdog_benefits_df.to_csv('benefits.csv')

C:\Users\cmlim\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
